In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print("Loading data...")
try:
    # Adjust paths as necessary
    train_df = pd.read_csv('train.csv', parse_dates=['date'])
    stores_df = pd.read_csv('stores.csv')
    holidays_df = pd.read_csv('holidays_events.csv', parse_dates=['date'])
    oil_df = pd.read_csv('oil.csv', parse_dates=['date'])
except FileNotFoundError as e:
    print(f"Error loading files. Ensure all CSVs are in the correct directory. Details: {e}")
    exit()

In [ ]:
# Data Cleaning and Merging 
print("Cleaning and merging data...")

# Standardize 'type' column name in holidays_df to avoid conflict
holidays_df.rename(columns={'type': 'holiday_type'}, inplace=True)

# Merge stores information with training data
df = train_df.merge(stores_df, on='store_nbr', how='left')

# Merge holidays information
# Note: Some dates have multiple holidays; the merge handles this.
df = df.merge(holidays_df, on='date', how='left')

In [ ]:
# Merge oil price information
# Backfill missing oil prices (often done in time series when price is constant)
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].fillna(method='ffill')
df = df.merge(oil_df, on='date', how='left')

# Fill NaNs created by the merge (e.g., if a day wasn't a holiday)
df['holiday_type'] = df['holiday_type'].fillna('None')
df['locale'] = df['locale'].fillna('None')

In [ ]:
# 3. Feature Engineering: Time-Based Features 
print("Creating time-based features...")

# Sort the data by store and date (CRITICAL for time series)
df = df.sort_values(by=['store_nbr', 'date']).reset_index(drop=True)

In [ ]:
# Extract Core Temporal Features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek # Monday=0, Sunday=6
df['dayofyear'] = df['date'].dt.dayofyear
df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)
df['weekend'] = (df['date'].dt.dayofweek >= 5).astype(int) # 1 if Saturday/Sunday

In [ ]:
# 4. Feature Engineering: Lags (For ML Models) 
# Lags help the model remember previous sales, usually on a store-item level.
print("Creating lag features...")

# Create 7-day lag for 'sales'
# We use a shift() grouped by 'store_nbr' and 'family' to ensure we only look at
# the history of that specific series.
df['sales_lag_7'] = df.groupby(['store_nbr', 'family'])['sales'].shift(7)

# Create 30-day rolling mean of 'sales'
df['sales_rolling_mean_30'] = df.groupby(['store_nbr', 'family'])['sales'].transform(
    lambda x: x.shift(7).rolling(30).mean()
)

In [ ]:
#  5. Feature Engineering: Categorical Encoding (For ML Models) 
print("Encoding categorical features...")

# Convert categorical columns to numerical using pd.get_dummies
categorical_cols = ['store_nbr', 'family', 'store_type', 'cluster', 'holiday_type', 'locale']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)

# --- 6. Final Data Preparation ---
# Drop the original 'date' column as we have extracted features from it
df.drop(['date', 'id'], axis=1, inplace=True)

In [ ]:
# 6. Final Data Preparation 
# Drop the original 'date' column as we have extracted features from it
df.drop(['date', 'id'], axis=1, inplace=True) 

# Handle NaNs from the lag/rolling features by dropping or imputing
# Dropping is common for the start of the time series where lags are undefined.
df.dropna(inplace=True) 

print("\n✅ Data Preparation Complete.")
print(f"Final dataset shape: {df.shape}")

In [ ]:
# 7. Define Features and Target (XGBoost)
X_xgb = df.drop('sales', axis=1)
y_xgb = df['sales']
print(f"XGBoost Feature Count: {X_xgb.shape[1]}")

In [ ]:
# 8. Time-Based Train/Test Split (XGBoost) 
# Use the first 90% of data for training, and the last 10% for testing.
split_point = int(len(X_xgb) * 0.9)
X_train_xgb, X_test_xgb = X_xgb.iloc[:split_point], X_xgb.iloc[split_point:]
y_train_xgb, y_test_xgb = y_xgb.iloc[:split_point], y_xgb.iloc[split_point:]

print(f"XGBoost Train/Test Split: {X_train_xgb.shape} / {X_test_xgb.shape}")